# 📊 Aadhaar Lifecycle Combined Insights
## UIDAI Hackathon 2026 - Stress & Behaviour Analysis

**Objective:** Connect enrolment, demographic, and biometric data to reveal the **living system** nature of Aadhaar.

---

### Key Connections:
- **High enrolment + High updates = System load hotspots**
- **Child enrolment today → Biometric spike in 5-10 years**
- **Migration states = Demographic + Biometric stress**

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['font.size'] = 11

## 1. Load All Three Datasets

In [ ]:
# Load Enrolment Data
enrol_path = Path('../data/enrolment/')
enrol_df = pd.concat([pd.read_csv(f) for f in enrol_path.glob('*.csv')], ignore_index=True)
enrol_df['date'] = pd.to_datetime(enrol_df['date'], format='%d-%m-%Y')
enrol_df['state'] = enrol_df['state'].str.strip().str.title()
enrol_df['total_enrolment'] = enrol_df['age_0_5'] + enrol_df['age_5_17'] + enrol_df['age_18_greater']

# Load Demographic Data
demo_path = Path('../data/demographic/')
demo_df = pd.concat([pd.read_csv(f) for f in demo_path.glob('*.csv')], ignore_index=True)
demo_df.columns = ['date', 'state', 'district', 'pincode', 'demo_age_5_17', 'demo_age_17_plus']
demo_df['date'] = pd.to_datetime(demo_df['date'], format='%d-%m-%Y')
demo_df['state'] = demo_df['state'].str.strip().str.title()
demo_df['total_demo_updates'] = demo_df['demo_age_5_17'] + demo_df['demo_age_17_plus']

# Load Biometric Data
bio_path = Path('../data/biometric/')
bio_df = pd.concat([pd.read_csv(f) for f in bio_path.glob('*.csv')], ignore_index=True)
bio_df.columns = ['date', 'state', 'district', 'pincode', 'bio_age_5_17', 'bio_age_17_plus']
bio_df['date'] = pd.to_datetime(bio_df['date'], format='%d-%m-%Y')
bio_df['state'] = bio_df['state'].str.strip().str.title()
bio_df['total_bio_updates'] = bio_df['bio_age_5_17'] + bio_df['bio_age_17_plus']

print("✅ All datasets loaded successfully!")
print(f"\n📊 Enrolment Records: {len(enrol_df):,}")
print(f"📊 Demographic Records: {len(demo_df):,}")
print(f"📊 Biometric Records: {len(bio_df):,}")

## 2. State-Level Aggregation & Merge

In [ ]:
# Aggregate by state
state_enrol = enrol_df.groupby('state').agg({
    'total_enrolment': 'sum',
    'age_0_5': 'sum',
    'age_5_17': 'sum',
    'age_18_greater': 'sum'
}).reset_index()

state_demo = demo_df.groupby('state').agg({
    'total_demo_updates': 'sum',
    'demo_age_5_17': 'sum',
    'demo_age_17_plus': 'sum'
}).reset_index()

state_bio = bio_df.groupby('state').agg({
    'total_bio_updates': 'sum',
    'bio_age_5_17': 'sum',
    'bio_age_17_plus': 'sum'
}).reset_index()

# Merge all datasets
combined = state_enrol.merge(state_demo, on='state', how='outer')
combined = combined.merge(state_bio, on='state', how='outer')
combined = combined.fillna(0)

# Calculate derived metrics
combined['total_system_load'] = combined['total_enrolment'] + combined['total_demo_updates'] + combined['total_bio_updates']
combined['update_intensity'] = (combined['total_demo_updates'] + combined['total_bio_updates']) / (combined['total_enrolment'] + 1)
combined['child_pipeline'] = combined['age_0_5'] + combined['age_5_17']  # Future biometric needs

print("✅ Combined dataset created!")
print(f"\nStates in combined data: {len(combined)}")
combined.head()

## 3. System Load Analysis

### The Big Picture: Which States Carry Maximum Aadhaar Lifecycle Load?

In [ ]:
# Top 15 states by total system load
top_load = combined.nlargest(15, 'total_system_load')

fig, ax = plt.subplots(figsize=(14, 9))

# Stacked bar chart
x = range(len(top_load))
width = 0.6

p1 = ax.barh(x, top_load['total_enrolment'], width, label='Enrolments', color='#3498DB')
p2 = ax.barh(x, top_load['total_demo_updates'], width, left=top_load['total_enrolment'], 
            label='Demographic Updates', color='#E74C3C')
p3 = ax.barh(x, top_load['total_bio_updates'], width, 
            left=top_load['total_enrolment'] + top_load['total_demo_updates'], 
            label='Biometric Updates', color='#2ECC71')

ax.set_yticks(x)
ax.set_yticklabels(top_load['state'])
ax.set_xlabel('Total System Load')
ax.set_title('🏆 AADHAAR TOTAL SYSTEM LOAD BY STATE (Enrol + Demo + Bio)', 
            fontsize=16, fontweight='bold')
ax.legend(loc='lower right', fontsize=11)
ax.invert_yaxis()

plt.tight_layout()
plt.savefig('../outputs/charts/combined_system_load.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 INSIGHT: Top states by total lifecycle load need comprehensive resource allocation")

In [ ]:
# System load composition
totals = {
    'Enrolments': combined['total_enrolment'].sum(),
    'Demographic Updates': combined['total_demo_updates'].sum(),
    'Biometric Updates': combined['total_bio_updates'].sum()
}

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart - Overall composition
colors = ['#3498DB', '#E74C3C', '#2ECC71']
explode = (0.02, 0.02, 0.02)
axes[0].pie(totals.values(), labels=totals.keys(), autopct='%1.1f%%',
           colors=colors, explode=explode, shadow=True, startangle=90,
           textprops={'fontsize': 11})
axes[0].set_title('📊 National Aadhaar Lifecycle Composition', fontsize=14, fontweight='bold')

# Bar chart with values
bars = axes[1].bar(totals.keys(), totals.values(), color=colors, edgecolor='black', linewidth=1.2)
axes[1].set_title('📊 Total Volume by Activity Type', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Total Count')
for bar, value in zip(bars, totals.values()):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01*max(totals.values()),
                f'{value/1e6:.1f}M', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/charts/combined_composition.png', dpi=150, bbox_inches='tight')
plt.show()

grand_total = sum(totals.values())
print("\n📊 NATIONAL AADHAAR LIFECYCLE SUMMARY:")
print("=" * 50)
print(f"TOTAL TRANSACTIONS: {grand_total:,.0f}")
for activity, count in totals.items():
    print(f"  {activity}: {count:,.0f} ({count/grand_total*100:.1f}%)")

## 4. Update Intensity Analysis

### Which states have high update activity relative to enrolments?

In [ ]:
# Update intensity: (Demo + Bio updates) / Enrolments
# Higher ratio = More update activity per enrolment = More active/mobile population

high_intensity = combined.nlargest(15, 'update_intensity')

fig, ax = plt.subplots(figsize=(14, 8))
colors = plt.cm.Spectral(np.linspace(0.2, 0.8, len(high_intensity)))

bars = ax.barh(high_intensity['state'], high_intensity['update_intensity'], color=colors, edgecolor='black')
ax.set_xlabel('Update Intensity (Updates per Enrolment)')
ax.set_title('🔥 UPDATE INTENSITY INDEX - States with High Update Activity per Enrolment', 
            fontsize=14, fontweight='bold')
ax.invert_yaxis()
ax.axvline(x=combined['update_intensity'].median(), color='red', linestyle='--', 
          linewidth=2, label=f"Median: {combined['update_intensity'].median():.2f}")
ax.legend()

for bar, value in zip(bars, high_intensity['update_intensity']):
    ax.text(value + 0.02, bar.get_y() + bar.get_height()/2, f'{value:.2f}x', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('../outputs/charts/combined_update_intensity.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 INTERPRETATION:")
print("   High update intensity = Active citizen engagement / Migration destinations")
print("   Low update intensity = Stable population / Lower digital penetration")

## 5. Child → Adult Transition Pipeline

### Future Biometric Demand = Today's Child Enrolments

In [ ]:
# States with highest child pipeline (future biometric pressure)
pipeline_states = combined.nlargest(15, 'child_pipeline')

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Left: Current child enrolments (pipeline)
colors1 = plt.cm.Oranges(np.linspace(0.4, 0.9, len(pipeline_states)))[::-1]
axes[0].barh(pipeline_states['state'], pipeline_states['child_pipeline'], color=colors1, edgecolor='black')
axes[0].set_xlabel('Child Enrolments (0-17 age group)')
axes[0].set_title('🔮 CHILD ENROLMENT PIPELINE\n(Future Biometric Demand)', fontsize=14, fontweight='bold')
axes[0].invert_yaxis()

# Right: Current biometric updates
colors2 = plt.cm.Blues(np.linspace(0.4, 0.9, len(pipeline_states)))[::-1]
axes[1].barh(pipeline_states['state'], pipeline_states['total_bio_updates'], color=colors2, edgecolor='black')
axes[1].set_xlabel('Current Biometric Updates')
axes[1].set_title('📊 CURRENT BIOMETRIC LOAD', fontsize=14, fontweight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.savefig('../outputs/charts/combined_child_pipeline.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n🔮 FUTURE DEMAND PRINCIPLE:")
print("   Today's child enrolments (0-17) = Tomorrow's mandatory biometric updates")
print("   States with high child pipelines need 5-10 year infrastructure planning")

In [ ]:
# Scatter plot: Child pipeline vs Current bio updates
fig, ax = plt.subplots(figsize=(12, 10))

scatter = ax.scatter(combined['child_pipeline'], combined['total_bio_updates'], 
                    s=combined['total_system_load']/10000, 
                    c=combined['update_intensity'], cmap='RdYlGn_r', 
                    alpha=0.7, edgecolors='black', linewidth=0.5)

# Add state labels for top states
top_states = combined.nlargest(10, 'total_system_load')
for _, row in top_states.iterrows():
    ax.annotate(row['state'], (row['child_pipeline'], row['total_bio_updates']),
               fontsize=9, fontweight='bold', 
               xytext=(5, 5), textcoords='offset points')

ax.set_xlabel('Child Enrolment Pipeline (0-17)', fontsize=12)
ax.set_ylabel('Current Biometric Updates', fontsize=12)
ax.set_title('🔮 CHILD PIPELINE vs CURRENT BIOMETRIC LOAD\n(Bubble size = Total System Load, Color = Update Intensity)', 
            fontsize=14, fontweight='bold')

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Update Intensity', fontsize=11)

# Add trend line
z = np.polyfit(combined['child_pipeline'], combined['total_bio_updates'], 1)
p = np.poly1d(z)
ax.plot(sorted(combined['child_pipeline']), p(sorted(combined['child_pipeline'])), 
       'r--', alpha=0.5, linewidth=2, label='Trend Line')
ax.legend(loc='upper left')

plt.tight_layout()
plt.savefig('../outputs/charts/combined_pipeline_scatter.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 READING THE CHART:")
print("   • Top-right quadrant: High current load + High future demand (PRIORITY)")
print("   • Large bubbles: High total system load")
print("   • Red color: High update intensity (active population)")

## 6. Correlation Analysis

In [ ]:
# Correlation matrix
corr_cols = ['total_enrolment', 'total_demo_updates', 'total_bio_updates', 
             'age_0_5', 'age_5_17', 'age_18_greater', 
             'child_pipeline', 'update_intensity']

corr_matrix = combined[corr_cols].corr()

# Plot heatmap
fig, ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdYlBu_r', 
           center=0, square=True, linewidths=1, ax=ax,
           cbar_kws={'shrink': 0.8})
ax.set_title('📊 CORRELATION MATRIX - Aadhaar Lifecycle Metrics', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/charts/combined_correlation.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n🔍 KEY CORRELATIONS:")
print("   • Enrolment ↔ Updates: Strong positive (high enrolment = high updates)")
print("   • Child pipeline ↔ Bio updates: Predictive relationship")
print("   • Update intensity varies independently (population behavior)")

## 7. Comprehensive State Ranking

In [ ]:
# Create a comprehensive ranking score
# Components: Total load, Update intensity, Future demand

combined['load_score'] = (combined['total_system_load'] / combined['total_system_load'].max()) * 100
combined['intensity_score'] = (combined['update_intensity'] / combined['update_intensity'].max()) * 100
combined['future_score'] = (combined['child_pipeline'] / combined['child_pipeline'].max()) * 100

# COMPOSITE AADHAAR PRIORITY INDEX
combined['priority_index'] = (
    combined['load_score'] * 0.40 +      # Current load (40%)
    combined['intensity_score'] * 0.30 +  # Update activity (30%)
    combined['future_score'] * 0.30       # Future demand (30%)
)

# Top 20 priority states
priority_states = combined.nlargest(20, 'priority_index')[['state', 'priority_index', 'load_score', 
                                                            'intensity_score', 'future_score',
                                                            'total_system_load']]

# Visualize
fig, ax = plt.subplots(figsize=(14, 10))

# Multi-component bar chart
x = range(len(priority_states))
width = 0.25

ax.barh([i-width for i in x], priority_states['load_score'], width, 
       label='Load Score (40%)', color='#3498DB', edgecolor='black')
ax.barh(x, priority_states['intensity_score'], width, 
       label='Intensity Score (30%)', color='#E74C3C', edgecolor='black')
ax.barh([i+width for i in x], priority_states['future_score'], width, 
       label='Future Score (30%)', color='#2ECC71', edgecolor='black')

ax.set_yticks(x)
ax.set_yticklabels(priority_states['state'])
ax.set_xlabel('Score (0-100)')
ax.set_title('🎯 AADHAAR PRIORITY INDEX - Component Scores by State', fontsize=14, fontweight='bold')
ax.legend(loc='lower right')
ax.invert_yaxis()

# Add priority index annotations
for i, (_, row) in enumerate(priority_states.iterrows()):
    ax.annotate(f"PI: {row['priority_index']:.0f}", 
               xy=(max(row['load_score'], row['intensity_score'], row['future_score']) + 5, i),
               fontsize=9, fontweight='bold', color='darkred')

plt.tight_layout()
plt.savefig('../outputs/charts/combined_priority_index.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Priority Index Table
print("\n" + "="*80)
print("🎯 AADHAAR PRIORITY INDEX - TOP 15 STATES")
print("="*80)
print(f"{'Rank':<6}{'State':<25}{'Priority Index':<18}{'Total Load':<15}")
print("-"*80)
for rank, (_, row) in enumerate(priority_states.head(15).iterrows(), 1):
    print(f"{rank:<6}{row['state']:<25}{row['priority_index']:<18.1f}{row['total_system_load']:,.0f}")

## 8. The Living System Visualization

In [ ]:
# Monthly trends across all three systems
enrol_monthly = enrol_df.groupby(enrol_df['date'].dt.to_period('M'))['total_enrolment'].sum().reset_index()
enrol_monthly.columns = ['month', 'enrolments']
enrol_monthly['month'] = enrol_monthly['month'].astype(str)

demo_monthly = demo_df.groupby(demo_df['date'].dt.to_period('M'))['total_demo_updates'].sum().reset_index()
demo_monthly.columns = ['month', 'demo_updates']
demo_monthly['month'] = demo_monthly['month'].astype(str)

bio_monthly = bio_df.groupby(bio_df['date'].dt.to_period('M'))['total_bio_updates'].sum().reset_index()
bio_monthly.columns = ['month', 'bio_updates']
bio_monthly['month'] = bio_monthly['month'].astype(str)

# Merge
monthly_all = enrol_monthly.merge(demo_monthly, on='month', how='outer')
monthly_all = monthly_all.merge(bio_monthly, on='month', how='outer')
monthly_all = monthly_all.fillna(0).sort_values('month')

# Visualization
fig, ax = plt.subplots(figsize=(16, 8))

x = range(len(monthly_all))
ax.plot(x, monthly_all['enrolments'], marker='o', linewidth=2.5, markersize=8, 
       label='Enrolments', color='#3498DB')
ax.plot(x, monthly_all['demo_updates'], marker='s', linewidth=2.5, markersize=8, 
       label='Demographic Updates', color='#E74C3C')
ax.plot(x, monthly_all['bio_updates'], marker='^', linewidth=2.5, markersize=8, 
       label='Biometric Updates', color='#2ECC71')

ax.fill_between(x, monthly_all['enrolments'], alpha=0.2, color='#3498DB')
ax.fill_between(x, monthly_all['demo_updates'], alpha=0.2, color='#E74C3C')
ax.fill_between(x, monthly_all['bio_updates'], alpha=0.2, color='#2ECC71')

ax.set_xticks(x)
ax.set_xticklabels(monthly_all['month'], rotation=45, ha='right')
ax.set_xlabel('Month-Year')
ax.set_ylabel('Transaction Volume')
ax.set_title('🌊 AADHAAR: THE LIVING SYSTEM\nEnrolment → Demographic → Biometric Lifecycle', 
            fontsize=16, fontweight='bold')
ax.legend(loc='upper right', fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../outputs/charts/combined_living_system.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n🌊 THE LIVING SYSTEM INSIGHT:")
print("   Aadhaar is not a one-time registration. It's a continuous lifecycle:")
print("   1. ENROLMENT → Initial entry into the system")
print("   2. DEMOGRAPHIC UPDATES → Life changes (migration, marriage, address)")
print("   3. BIOMETRIC UPDATES → Biological changes (child→adult, aging)")

## 9. Final Summary Tables

In [ ]:
# Create comprehensive final summary
final_summary = combined[['state', 'total_enrolment', 'total_demo_updates', 'total_bio_updates',
                          'total_system_load', 'update_intensity', 'child_pipeline', 'priority_index']].copy()

final_summary.columns = ['State', 'Enrolments', 'Demo_Updates', 'Bio_Updates', 
                         'Total_Load', 'Update_Intensity', 'Child_Pipeline', 'Priority_Index']

final_summary = final_summary.sort_values('Priority_Index', ascending=False)
final_summary['Priority_Index'] = final_summary['Priority_Index'].round(1)
final_summary['Update_Intensity'] = final_summary['Update_Intensity'].round(2)

# Save
final_summary.to_csv('../outputs/summary_tables/combined_state_summary.csv', index=False)
print("✅ Combined state summary saved!")

# Display
print("\n📊 COMBINED STATE SUMMARY (Top 15):")
final_summary.head(15)

## 10. Executive Summary & Policy Recommendations

In [ ]:
# Generate executive summary
print("\n" + "="*80)
print("📋 EXECUTIVE SUMMARY - AADHAAR LIFECYCLE STRESS & BEHAVIOUR ANALYSIS")
print("="*80)

total_enrol = combined['total_enrolment'].sum()
total_demo = combined['total_demo_updates'].sum()
total_bio = combined['total_bio_updates'].sum()
grand_total = total_enrol + total_demo + total_bio

print(f"""
┌─────────────────────────────────────────────────────────────────────────┐
│                    NATIONAL AADHAAR LIFECYCLE METRICS                   │
├─────────────────────────────────────────────────────────────────────────┤
│  Total Enrolments:          {total_enrol:>15,.0f}  ({total_enrol/grand_total*100:>5.1f}%)       │
│  Demographic Updates:       {total_demo:>15,.0f}  ({total_demo/grand_total*100:>5.1f}%)       │
│  Biometric Updates:         {total_bio:>15,.0f}  ({total_bio/grand_total*100:>5.1f}%)       │
├─────────────────────────────────────────────────────────────────────────┤
│  GRAND TOTAL:               {grand_total:>15,.0f}  (100.0%)       │
├─────────────────────────────────────────────────────────────────────────┤
│  States/UTs Covered:        {combined['state'].nunique():>15}                       │
│  Child Pipeline (0-17):     {combined['child_pipeline'].sum():>15,.0f}                       │
└─────────────────────────────────────────────────────────────────────────┘
""")

# Top priority states
top_5 = final_summary.head(5)['State'].tolist()
print(f"🎯 TOP 5 PRIORITY STATES: {', '.join(top_5)}")

# High future demand
high_future = combined.nlargest(5, 'child_pipeline')['state'].tolist()
print(f"\n🔮 HIGH FUTURE DEMAND STATES: {', '.join(high_future)}")

In [ ]:
print("""
================================================================================
📌 STRATEGIC POLICY RECOMMENDATIONS
================================================================================

🔷 IMMEDIATE ACTIONS (0-1 Year)
─────────────────────────────────
1. INFRASTRUCTURE SCALING
   • Deploy additional enrolment centers in top priority states
   • Upgrade biometric capture devices to latest technology
   • Extend operational hours during peak periods

2. STAFF AUGMENTATION
   • Recruit and train operators for high-load districts
   • Implement performance-based incentives
   • Cross-train staff for multi-function capability

3. DIGITAL ENABLEMENT
   • Promote self-service demographic updates via mobile app
   • Reduce physical center dependency for simple changes
   • Implement appointment-based scheduling

🔷 MEDIUM-TERM INITIATIVES (1-3 Years)
────────────────────────────────────────
1. CHILD TRANSITION MANAGEMENT
   • School-based biometric update camps (Class 10/12)
   • Pre-scheduled appointments for turning-18 cohort
   • Hospital partnerships for birth registration + Aadhaar

2. MIGRATION CORRIDOR FOCUS
   • Special camps at industrial zones, construction sites
   • Interstate data sync for seamless updates
   • Mobile units for transient worker populations

3. QUALITY IMPROVEMENT
   • Reduce biometric capture failures
   • Implement real-time quality checks
   • Regular equipment calibration schedules

🔷 LONG-TERM STRATEGY (3-10 Years)
────────────────────────────────────
1. PREDICTIVE PLANNING
   • Use child enrolment data to forecast biometric demand
   • State-wise infrastructure investment roadmaps
   • Dynamic resource allocation algorithms

2. TECHNOLOGY EVOLUTION
   • Explore contactless biometric capture
   • AI-assisted identity verification
   • Blockchain for audit trail integrity

3. CITIZEN-CENTRIC DESIGN
   • Minimize visits for lifecycle events
   • Proactive reminders for mandatory updates
   • Multilingual, accessible interfaces

================================================================================
🌟 CORE INSIGHT: Aadhaar is a LIVING SYSTEM, not a static database.
   Success lies in anticipating and managing this continuous lifecycle.
================================================================================
""")

In [ ]:
# Save executive summary
summary_text = f"""
AADHAAR LIFECYCLE STRESS & BEHAVIOUR ANALYSIS
UIDAI HACKATHON 2026 - EXECUTIVE SUMMARY
================================================

NATIONAL METRICS (as of {pd.Timestamp.now().strftime('%B %Y')})
------------------------------------------------
Total Enrolments:        {total_enrol:,.0f}
Demographic Updates:     {total_demo:,.0f}
Biometric Updates:       {total_bio:,.0f}
GRAND TOTAL:             {grand_total:,.0f}

GEOGRAPHIC COVERAGE
------------------------------------------------
States/UTs:              {combined['state'].nunique()}
Child Pipeline (0-17):   {combined['child_pipeline'].sum():,.0f}

TOP 5 PRIORITY STATES (by Priority Index)
------------------------------------------------
{chr(10).join([f'{i+1}. {s}' for i, s in enumerate(top_5)])}

HIGH FUTURE DEMAND STATES (by Child Pipeline)
------------------------------------------------
{chr(10).join([f'{i+1}. {s}' for i, s in enumerate(high_future)])}

KEY FINDINGS
------------------------------------------------
1. Aadhaar is a living system with continuous lifecycle activity
2. Demographic updates dominate update volume (migration/mobility)
3. Child enrolments today predict biometric demand in 5-10 years
4. Top states carry disproportionate infrastructure burden
5. Update intensity varies by state (population behavior)

RECOMMENDATIONS
------------------------------------------------
1. Scale infrastructure in high-priority states
2. Plan for child→adult biometric transition wave
3. Enable digital self-service for simple updates
4. Deploy mobile units in migration corridors
5. Implement predictive resource allocation

Report Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}
"""

with open('../outputs/summary_tables/executive_summary.txt', 'w') as f:
    f.write(summary_text)

print("✅ Executive summary saved to outputs/summary_tables/executive_summary.txt")

---
## 📊 Analysis Complete!

### Generated Outputs:

**Charts (outputs/charts/):**
- combined_system_load.png
- combined_composition.png
- combined_update_intensity.png
- combined_child_pipeline.png
- combined_pipeline_scatter.png
- combined_correlation.png
- combined_priority_index.png
- combined_living_system.png

**Summary Tables (outputs/summary_tables/):**
- combined_state_summary.csv
- executive_summary.txt

---
### 🎯 Key Message for UIDAI Jury:

> **"Aadhaar is not a static ID database. It's a LIVING SYSTEM that continuously evolves through citizen lifecycle events. Our analysis reveals the stress points, predicts future demand, and provides actionable policy recommendations for sustainable system management."**